In [1]:
%pip install --upgrade --quiet python-dotenv langchain langchain-google-genai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 6.4 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google API Key: ")


Enter your Google API Key: ··········


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0
)


In [4]:
task = "Write a rejection email to a candidate."

print("--- LAZY PROMPT ---")
print(llm.invoke(task).content)


--- LAZY PROMPT ---
Here are a few options for a rejection email, ranging from a standard template to one for a candidate who interviewed. Choose the one that best fits your situation.

---

**Option 1: Standard Rejection (No Interview)**

This is suitable for candidates who applied but were not selected for an interview.

**Subject: Update on Your Application for [Job Title] at [Company Name]**

Dear [Candidate Name],

Thank you for your interest in the [Job Title] position at [Company Name] and for taking the time to submit your application.

We received a large number of highly qualified applications for this role. While your qualifications are impressive, we have decided to move forward with other candidates whose profiles were a closer match for the specific requirements of this position at this time.

We appreciate you considering [Company Name] as a potential employer and wish you the best of luck in your job search and future endeavors.

Sincerely,

[Your Name]
[Your Title]
[Co

In [5]:
structured_prompt = """
# Context
You are an HR Manager at a quirky startup called 'RocketBoots'.

# Objective
Write a rejection email to a candidate named Bob.

# Constraints
1. Be extremely brief (under 50 words).
2. Do NOT say 'we found someone better'. Say 'the role changed'.
3. Sign off with 'Keep flying'.

# Output Format
Plain text, no subject line.
"""

print("--- STRUCTURED PROMPT ---")
print(llm.invoke(structured_prompt).content)


--- STRUCTURED PROMPT ---
Hi Bob,

Thank you for your interest in RocketBoots. We appreciate your time and effort.

While your application was impressive, the requirements for this role have recently changed. We won't be moving forward with your candidacy at this time.

Keep flying,
RocketBoots HR


In [6]:
assignment_prompt = """
# Context
You are a Senior Python Developer.

# Objective
Write a Python function that reverses a string.

# Constraints
1. You must use recursion.
2. You are NOT allowed to use slicing (no [::-1]).
3. Include detailed docstrings explaining how the recursion works.

# Output Format
Only provide the Python function.
"""

print(llm.invoke(assignment_prompt).content)


```python
def reverse_string_recursive(s: str) -> str:
    """
    Reverses a given string using recursion.

    This function takes a string and reverses it by recursively
    deconstructing the string and rebuilding it in reverse order.
    It adheres to the constraint of not using the `[::-1]` slicing syntax.

    The recursion works as follows:
    1.  **Base Case**:
        If the input string `s` is empty (`""`) or contains only one character
        (e.g., `"a"`), it is already considered reversed. In this scenario,
        the function returns the string `s` itself. This is the crucial
        stopping condition that prevents infinite recursion.

    2.  **Recursive Step**:
        For any string `s` that has two or more characters:
        a.  It identifies the *last character* of the string (`s[-1]`).
        b.  It creates a *substring* consisting of all characters *except* the last one (`s[:-1]`).
        c.  It then makes a recursive call to `reverse_string_recursive` with

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.5
)


In [8]:
prompt_zero = "Combine 'Angry' and 'Hungry' into a funny new word."
print("Zero-Shot:", llm.invoke(prompt_zero).content)


Zero-Shot: The most common and widely accepted funny word for that is:

**Hangry**

It's a perfect blend and perfectly describes that irritable feeling when you're hungry!


In [9]:
prompt_few = """
Combine words into a funny new word. Give a sarcastic definition.

Input: Breakfast + Lunch
Output: Brunch (An excuse to drink alcohol before noon)

Input: Chill + Relax
Output: Chillax (What annoying people say when you are having a panic attack)

Input: Angry + Hungry
Output:
"""

print("Few-Shot:", llm.invoke(prompt_few).content)


Few-Shot: Output: Hangry (The official medical diagnosis for why you're being an insufferable jerk until someone feeds you.)


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


In [11]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Example database
examples = [
    {"input": "The internet is down.", "output": "We are observing connectivity latency."},
    {"input": "This code implies a bug.", "output": "The logic suggests unintended behavior."},
    {"input": "I hate this feature.", "output": "This feature does not align with my preferences."},
]

# Template for one example
example_fmt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# Few-shot container
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_fmt,
    examples=examples
)

# Final prompt
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a Corpo-Speak Translator. Rewrite the input to sound professional."),
    few_shot_prompt,
    ("human", "{text}")
])

chain = final_prompt | llm

print(chain.invoke({"text": "This app sucks."}).content)


The application's current iteration presents opportunities for significant enhancement.


In [12]:
#ASSIGNMENT
# ===== Assignment: Part 2a – Structured Prompt for Code Generation =====

structured_prompt = """
# Context
You are a Senior Python Developer with strong software engineering standards.

# Objective
Write a Python function that reverses a string.

# Constraints
1. You MUST use recursion.
2. Do NOT use slicing (no [::-1]).
3. The function must handle empty strings correctly.

# Style
1. Include a detailed Python docstring.
2. Follow clean, readable Python conventions.

# Output Format
Provide only valid Python code.
"""

print(llm.invoke(structured_prompt).content)

```python
def reverse_string(s: str) -> str:
    """
    Reverses a given string using a recursive approach without utilizing string slicing
    for the reversal operation itself.

    This function takes an input string and returns a new string where the characters
    are in the reverse order. It strictly adheres to a recursive implementation
    and avoids Python's convenient string slicing for direct reversal (e.g., `s[::-1]`).

    Args:
        s: The input string to be reversed. This can be an empty string,
           a string with a single character, or a longer string.

    Returns:
        A new string that is the reverse of the input string `s`.

    Constraints Met:
    1.  **Uses recursion:** The function calls itself to solve smaller subproblems.
    2.  **Does NOT use slicing (no [::-1]):** The specific `[::-1]` slice used for
        reversal is avoided. While `s[1:]` is used to obtain a substring for the
        recursive call, this is a standard substring operation an